In [53]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
module_path2 = os.path.abspath(os.path.join('../../AxolotlDataScience'))

if module_path not in sys.path:
    sys.path.append(module_path)
    
if module_path2 not in sys.path:
    sys.path.append(module_path2)

from DataManipulation import DataManipulation as axolotlManipulation
from DataVisualization import TypeOfAccessBarChart
from Graphics import MultipleColumnsBarChartByClass
from Graphics import SingleColumnBarChartByClass
from MachineLearning.Classifiers import ClassifierFactory
from MachineLearning.Classifiers import ClassifierTypes
from Validation import NFoldCrossValidation as validator
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

data_frame = pd.read_csv('../input_data/ge_mes_olimpiadas.csv', sep=',')

Churn e Recomendações
----------------------


Na base de dados do globo esporte temos 3 tipos de assinaturas, os que possuem conta no cartola, os que possuem o catola pro, e finalmente os assinantes da globo.com. Para tais usuários é importante saber se eles pretendem sair, e para os não assinantes, é importante indentificar os possíveis assinantes para axiliar o time de marketing em sua tomada de decisão.

Para a identificação de churn e recomendação de assinatura iremos utilizar métodos de classificação e/ou clusterização, e trabalhar em cima dos falso positivos e falso negativos, ou seja:

* Falso positivo: Se um usuário é classificado como assinante, porém ele não possui assinatura, ele é um caso de possível assinante, pois apesar de não possuir assinatura possui o mesmo comportamento que os que a tem.

* Falso negativo: Se um usuário é classificado como não-assinante, porém ele é um assinante, ele é um caso de churn, pois está cada vez mais se comportando como não assinante.


### Cartola

Apesar de não ser uma assinatura paga, o cartola pode gerar interesse nos usuários de ter uma conta cartola-pro, sendo assim é interessante captar e manter os usuários do cartola, por este motivo também iremos analisar essa classe de assinantes.

O cartola é sobre futebol nacional, sendo assim informações sobre outras modalidades olímpicas senão futebol são irrelevantes para este tipo de usuário, quanto a informações referentes a outras modalidades de futebol, inicialmente iremos mante-las.

#### Tempo de Acesso

Nós além das informações dos usuários temos os tempos que os usuários passaram em noticias de um determinado assunto, deste modo iremos fazer algumas modificações nesses dados:

* Ao invés do tempo passado em cada assunto, iremos ver qual a porcentagem de tempo que ele passou em cada assunto.
* Normalizaremos este número com a média.
* Criaremos classes de interesses.

In [ ]:
cartola_df = data_frame.copy()

axolotl_manipulation = axolotlManipulation.DataManipulation(cartola_df)

to_remove_columns = ["home_notic_olimp", "ginastica", "basquete", "volei", "handebol", "tenis", "atletismo", "natacao", "judo",
                     "saltos_orn", "canoagem", "home", "cartola_pro", "tipo", "user", "dias_desktop", "pviews_desktop", "visitas_desktop", "dias_mobile", "pviews_mobile",
                     "visitas_mobile", 'tempo_total_desktop', 'tempo_total_mobile']

age_range = [-np.inf, 0.1, 13, 23, 35, 48, np.inf]
dias_range = axolotl_manipulation.get_discretization_intervals_based_on_number_of_groups('dias', 5)
pviews_range = axolotl_manipulation.get_discretization_intervals_based_on_number_of_groups('pviews', 10)
visitas_range = axolotl_manipulation.get_discretization_intervals_based_on_number_of_groups('visitas', 10)
tempo_total_range = axolotl_manipulation.get_discretization_intervals_based_on_number_of_groups('tempo_total', 10)

cartola_df = axolotl_manipulation.drop_columns(to_remove_columns) \
                                 .set_categorical_columns(['sexo', 'uf']) \
                                 .divide_by_column(['fut_int', 'futebol', 'fut_olimp', 'blog_cartola'], 'tempo_total') \
                                 .normalize_columns(['fut_int_by_tempo_total', 'futebol_by_tempo_total', 'fut_olimp_by_tempo_total', 'blog_cartola_by_tempo_total'])\
                                 .discretize_column('idade', age_range) \
                                 .discretize_column('dias', dias_range) \
                                 .discretize_column('pviews', pviews_range) \
                                 .discretize_column('visitas', visitas_range) \
                                 .discretize_column('tempo_total', tempo_total_range)

cartola_df = axolotl_manipulation.get_data_frame()
cartola_df['cartola'].fillna(0)
cartola_df['idade'].fillna(0, inplace=True)

classifier_types = ClassifierTypes.ClassifierTypes
factory = ClassifierFactory.ClassifierFactory()
attributes = cartola_df.loc[:, cartola_df.columns != 'cartola'].columns.values.tolist()
learning_algorithm = factory.choose_classifier(classifier_types.SVM).init_learning_algorithm() \
                                                                           .get_learning_algorithm()

learning_algorithm.fit(cartola_df.loc[:, cartola_df.columns != 'cartola'], cartola_df["cartola"])

n_fold = validator.NFoldCrossValidation(learning_algorithm, cartola_df.loc[:, cartola_df.columns != 'cartola'], cartola_df["cartola"])
accuracy = n_fold.ten_fold_cross_validation()

print(accuracy)